# 要約 
このJupyter Notebookは、Kaggleの「20の質問」ゲーム用のエージェントをテストするためのコードを提供しています。具体的には、メモリ制限のために`kaggle_environments`を使用しない方法で、質問者（asker）と回答者（answerer）エージェントを実装しています。

### 問題の概要
ノートブックは、言語モデルを用いて「20の質問」ゲームをどのようにプレイするかを検証しています。このゲームは、質問者が1つのキーワードを推測するために、回答者から「はい」または「いいえ」で答えられる質問を投げかける形式で進行します。エージェントは、できるだけ少ない質問で正解にたどり着くことを目指します。

### 手法とライブラリ
1. **ライブラリ**:
   - `transformers`: 特に`AutoTokenizer`と`AutoModelForCausalLM`クラスを使用して事前訓練済みの言語モデルを初期化します。
   - `torch`: モデルの処理をGPU上で行うために使用します。

2. **エージェントの設計**:
   - エージェントは、インスタンス`Robot`から構成されています。このクラスは、質問する、応答する、推測するという異なる役割を持つメソッドを提供しています。
   - `generate_answer`関数は、トークナイザーを使用してテキストをトークン化し、モデルに渡して出力を生成します。

3. **ゲームの進行**:
   - `Observation`クラスを使ってゲームの状態（現在のターン、役割、キーワードなど）を管理します。
   - 各ラウンドにおいて、エージェントは質問、回答、推測を行い、その結果を表示します。

4. **キーワードとカテゴリ**:
   - 特定のキーワードとそれに関連するカテゴリをJSONデータから取得し、データフレームに変換します。

このノートブック全体を通じて、エージェントは「20の質問」ゲームをプレイする能力を持ち、最終的には与えられたキーワードを推測することに挑戦します。

---


# 用語概説 
以下に、Jupyter Notebookの内容に関連し、機械学習・深層学習の初心者がつまずきそうな専門用語の解説を列挙します。

1. **kaggle_environments**:
   - Kaggleの環境でエージェントの訓練やテストを行うためのライブラリです。コンペティションのルールや動作をシミュレーションするために用いられます。このノートブックでは、メモリ制限のために使用しないとされています。

2. **エージェント**:
   - ゲームやシミュレーションにおいて、行動を取る主体のことを指します。この文脈では、質問者と回答者の役割を持つ二つのエージェントを指し、相互に質問と回答を行います。

3. **トークナイザー**:
   - 自然言語処理(NLP)において、テキストデータをトークンと呼ばれる単位に分割するツールです。このトークンは通常、単語やサブワードの単位を持ち、モデルが理解できる形に変換する役割を果たします。

4. **モデルID**:
   - 特定の事前学習済みモデルを指し、モデルのパラメータやアーキテクチャを特定するための識別子です。このノートブックでは、エージェントが使用するモデルを指定しています。

5. **CUDA**:
   - NVIDIAのGPUを用いて計算を行うための API で、高速化のために利用されます。Deep Learningの訓練や推論を効率化するために用いられます。

6. **メモリ効率的なSDP (Stochastic Dynamic Programming)**:
   - モデルのメモリ使用量を削減するための手法です。深層学習において、大きなモデルを扱う際に重要な役割を果たします。この特定の文脈では、CUDAバックエンドで動作するメモリ効率的なアルゴリズムを無効にしています。

7. **torch_dtype**:
   - PyTorchにおいて、テンソルのデータ型を指定するための引数です。このノートブックでは、メモリ効率向上のために `torch.bfloat16` 型が指定されています。

8. **特別なトークン (EOTトークン)**:
   - モデルがトークンの生成を制御するために使用される特別なシンボルです。ここでは、会話の終了を示すために利用されています。

9. **出力生成 (generate_answer)**:
   - モデルが入力情報に基づいて出力を生成するプロセスです。この関数は、ユーザーの質問に対するモデルの回答を生成し、適切な形式にデコードします。

10. **状態 (obs)**:
    - ゲームの進行状況やエージェントが参照する情報が含まれたオブジェクトです。このノートブックでは、質問、回答、推測などの情報を含むクラス`Observation`として定義されています。

11. **JSONデータ**:
    - JavaScript Object Notationの略で、データを構造化するためのフォーマットです。このノートブックでは、キーワードやそのカテゴリに関する情報の保存とアクセスに使われています。

12. **データフレーム (DataFrame)**:
    - 複数のデータを表形式で扱うためのデータ構造で、Pandasライブラリの特徴的な構成です。このノートブックでは、キーワード、カテゴリ、同義語を整理するために使用されています。

初心者の方は、専門用語の理解を深めることで、コードの動作や目的を把握しやすくなるでしょう。このような用語に慣れていくことで、機械学習や深層学習の文脈における理解が進むはずです。

---


# kaggle_environmentsを使用せずにエージェントをテストすることができます
kaggle_environmentsを使用することはメモリの制限によりできません。
そのため、質問者および回答者エージェントをテストするためのコードを作成しました。  

このエージェントは以下に基づいて作成されました：  
[llama3-8b- LLM20 Questions [LB 750]](https://www.kaggle.com/code/wouldyoujustfocus/llama3-8b-llm20-questions-lb-750)


In [ ]:
%%bash
mkdir -p /kaggle/working/submission

In [ ]:
%%writefile -a submission/main.py

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from kaggle_secrets import UserSecretsClient
import os
import sys
import shutil

torch.backends.cuda.enable_mem_efficient_sdp(False)  # メモリ効率的なSDPを無効にします
torch.backends.cuda.enable_flash_sdp(False)  # フラッシュSDPを無効にします



KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    model_id = os.path.join(KAGGLE_AGENT_PATH, "1")  # エージェントパスが存在する場合のモデルID
else:
    model_id = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"  # デフォルトのモデルID


tokenizer = AutoTokenizer.from_pretrained(model_id)  # トークナイザーの初期化
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")  # モデルの初期化
id_eot = tokenizer.convert_tokens_to_ids(["<|eot_id|>"])[0]  # モデルで使用する特別なトークンのIDを取得


def generate_answer(template):
    inp_ids = tokenizer(template, return_tensors="pt").to("cuda")  # 入力テンプレートをトークン化してCUDAに転送
    out_ids = model.generate(**inp_ids,max_new_tokens=15).squeeze()  # モデルから出力を生成
    start_gen = inp_ids.input_ids.shape[1]  # 入力のサイズを取得
    out_ids = out_ids[start_gen:]  # 出力から入力のサイズを引く
    if id_eot in out_ids:  # 特殊なトークンが出力に含まれる場合
        stop = out_ids.tolist().index(id_eot)  # 特殊なトークンのインデックスを取得
        out = tokenizer.decode(out_ids[:stop])  # 特殊なトークンまでデコード
    else:
        out = tokenizer.decode(out_ids)  # デコード
    return out  # 生成された出力を返す
    

class Robot:
    def __init__(self):
        pass
    
    def on(self, mode, obs):
        assert mode in ["asking", "guessing", "answering"], "mode can only take one of these values: asking, answering, guessing"
        if mode == "asking":
            # 質問者役を起動します
            output = self.asker(obs)
        if mode == "answering":
            # 回答者役を起動します
            output = self.answerer(obs)
            if "yes" in output.lower() or "Yes" in output.lower():
                output = "yes"  # 回答が「はい」の場合
            elif "no" in output.lower() or "No" in output.lower():
                output = "no"  # 回答が「いいえ」の場合
            else:
                output = "yes"  # 不明な場合は「はい」とします
        if mode == "guessing":
            # 推測者役を起動します
            output = self.asker(obs)  # 質問者の機能を呼び出します
        return output
    
    
    def asker(self, obs):
        sys_prompt = """
        あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。
        このゲームでは、回答者がキーワードを考え、質問者がイエスまたはノーの質問に答えます。
        キーワードは特定の場所や物です。\n
        """
    
        if obs.turnType =="ask":
            ask_prompt = sys_prompt + """
            キーワードがモロッコであると仮定した場合、これはどう機能するかの例を示します：
            例：
            <あなた：これは場所ですか？
            ユーザー：はい
            あなた：ヨーロッパにありますか？
            ユーザー：いいえ
            あなた：アフリカにありますか？
            ユーザー：はい
            あなた：そこに住んでいるほとんどの人は肌が暗いですか？
            ユーザー：いいえ
            あなた：mで始まる国名ですか？
            ユーザー：はい
            あなた：それはモロッコですか？
            ユーザー：はい。>

            ユーザーは単語を選択しました。最初の質問をしてください！
            短く簡潔に、1つの質問のみで、余分な言葉は使わないでください！"""
            chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{ask_prompt}<|eot_id|>"""  # チャットのテンプレートを作成
            chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
            if len(obs.questions)>=1:
                for q, a in zip(obs.questions, obs.answers):  # 質問と回答を連結
                    chat_template += f"{q}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
                    chat_template += f"{a}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
                    
        elif obs.turnType == "guess":
            conv = ""
            for q, a in zip(obs.questions, obs.answers):  # 過去の質問と回答を取得
                conv += f"""質問: {q}\n回答: {a}\n"""
            guess_prompt =  sys_prompt + f"""
            現在のゲームの状態は次のとおりです:\n{conv}
            会話に基づいて、単語を推測してください。余分な説明はしないで、単語のみを答えてください。
            """
            chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{guess_prompt}<|eot_id|>"""  # 推測用のテンプレートを作成
            chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
                
        output = generate_answer(chat_template)        
        return output
        
        
        
    def answerer(self, obs):
        sys_prompt = f"""
        あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを考え、質問者がイエスまたはノーの質問に答えます。
        キーワードは特定の場所や物です。\n
        ユーザーの質問を理解できるようにし、あなたがプレイしているキーワードを理解してください。
        現在、ユーザーが推測すべき単語は: "{obs.keyword}"、カテゴリは"{obs.category}"です。
        キーワードがモロッコでカテゴリが「場所」であると仮定した場合、これはどう機能するかの例を示します：
        例：
        <ユーザー：これは場所ですか？
        あなた：はい
        ユーザー：ヨーロッパにありますか？
        あなた：いいえ
        ユーザー：アフリカにありますか？
        あなた：はい
        ユーザー：そこに住んでいるほとんどの人は肌が暗いですか？
        あなた：いいえ
        ユーザー：mで始まる国名ですか？
        あなた：はい
        ユーザー：それはモロッコですか？
        あなた：はい。>
        """
        
        chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{sys_prompt}<|eot_id|>"""  # チャットのテンプレートを作成
        chat_template += "<|start_header_id|>user<|end_header_id|>\n\n"
        chat_template += f"{obs.questions[0]}<|eot_id|>"
        chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
        if len(obs.answers)>=1:
            for q, a in zip(obs.questions[1:], obs.answers):  # 各質問と回答を連結
                chat_template += f"{a}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
                chat_template += f"{q}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        output = generate_answer(chat_template)  # モデルに送信して出力を生成
        return output
    
    
robot = Robot()


def agent(obs, cfg):
    
    if obs.turnType =="ask":  # 質問するターン
        response = robot.on(mode = "asking", obs = obs)
        
    elif obs.turnType =="guess":  # 推測するターン
        response = robot.on(mode = "guessing", obs = obs)
        
    elif obs.turnType =="answer":  # 回答するターン
        response = robot.on(mode = "answering", obs = obs)
        
    if response == None or len(response)<=1:  # レスポンスが不正な場合
        response = "yes"  # デフォルトのレスポンスを設定
        
    return response  # 最終レスポンスを返す

## 以下のコードを main.py の後に貼り付けてください


In [ ]:
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py

In [ ]:
import json
import pandas as pd
from submission.main import agent
from keywords_local import KEYWORDS_JSON

class Observation:
    def __init__(self):
        self.step = 0
        self.role = "guesser"  # 最初の役割は推測者
        self.turnType = "ask"  # 初期ターンは質問
        self.keyword = "Japan"  # デフォルトのキーワード
        self.category = "country"  # デフォルトのカテゴリ
        self.questions = []  # 質問のリスト
        self.answers = []  # 回答のリスト
        self.guesses = []  # 推測のリスト
        
def create_keyword_df(KEYWORDS_JSON):
    json_data = json.loads(KEYWORDS_JSON)  # JSONデータを読み込む

    keyword_list = []  # キーワードのリスト
    category_list = []  # カテゴリのリスト
    alts_list = []  # 同義語リスト

    for i in range(len(json_data)):
        for j in range(len(json_data[i]['words'])):  # 各単語をループ
            keyword = json_data[i]['words'][j]['keyword']
            keyword_list.append(keyword)  # キーワードを追加
            category_list.append(json_data[i]['category'])  # カテゴリを追加
            alts_list.append(json_data[i]['words'][j]['alts'])  # 同義語を追加

    data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])  # データフレームを作成
    data_pd['keyword'] = keyword_list  # キーワードカラム
    data_pd['category'] = category_list  # カテゴリカラム
    data_pd['alts'] = alts_list  # 同義語カラム
    
    return data_pd  # データフレームを返す
    
keywords_df = create_keyword_df(KEYWORDS_JSON)  # データフレームを作成

In [ ]:
obs = Observation()  # 観察インスタンスの作成
cfg = "_"

sample_df = keywords_df.sample()  # ランダムにサンプルを取得
obs.keyword = sample_df["keyword"].values[0]  # サンプルのキーワードを設定
obs.category = sample_df["category"].values[0]  # サンプルのカテゴリを設定
alts_list = sample_df["alts"].values[0]  # 同義語リストを取得
alts_list.append(obs.keyword)  # キーワードを同義語リストに追加

print(f"keyword:{obs.keyword}")  # キーワードを出力

for round in range(20):  # 最大20ラウンドをループ
    obs.step = round+1  # ステップを更新
    
    obs.role = "guesser"  # 現在の役割を推測者に設定
    obs.turnType = "ask"  # 現在のターンを質問に設定
    question = agent(obs, cfg)  # 質問を取得
    obs.questions.append(question)  # 質問をリストに追加
    
    obs.role = "answerer"  # 現在の役割を回答者に設定
    obs.turnType = "answer"  # 現在のターンを回答に設定
    answer = agent(obs, cfg)  # 回答を取得
    obs.answers.append(answer)  # 回答をリストに追加
    
    obs.role = "guesser"  # 再び推測者の役割に戻す
    obs.turnType = "guess"  # 現在のターンを推測に設定
    guess = agent(obs, cfg)  # 推測を取得
    obs.guesses.append(guess)  # 推測をリストに追加
    
    print(f"round: {round+1}")  # ラウンド番号を出力
    print(f"question: {question}")  # 質問を出力
    print(f"answer: {answer}")  # 回答を出力
    print(f"guess: {guess}")  # 推測を出力
    
    if guess in alts_list:  # 推測が同義語リストに含まれる場合
        print("勝ちました!!")  # 勝利メッセージを出力
        break